<a href="https://colab.research.google.com/github/VenGr0/Convolutional_NN/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%9618.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications import MobileNet
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras import optimizers

def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=predictions)

In [ ]:
model_maker.summary()

In [ ]:
from keras.applications import MobileNet
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras import optimizers
from tensorflow.keras import Input, Model

IMG_WIDTH=224
IMG_HEIGHT=224

NUM_CLASSES=2

def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_HEIGHT, IMG_WIDTH, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=predictions)

In [ ]:
animal = model_maker()
animal

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
# Разархивируем датасета во временную папку 'temp'
!unzip -qo "cat-and-dog" -d ./temp

--2024-05-14 14:42:05--  https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228082266 (218M) [application/x-zip-compressed]
Saving to: ‘cat-and-dog.zip’

cat-and-dog.zip     100%[===================>] 217.52M  19.6MB/s    in 12s     

2024-05-14 14:42:18 (17.9 MB/s) - ‘cat-and-dog.zip’ saved [228082266/228082266]



In [ ]:
import os

import shutil
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

# Папка с сортировкой по категориям
IMAGE_PATH = './temp/training_set/training_set/'

# Папка с выборками
BASE_DIR = './dataset/'

# Определение списка имен классов
CLASS_LIST = sorted(os.listdir(IMAGE_PATH))

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)

IMAGE_PATH_1 = './temp/test_set/test_set/'

# При повторном запуске пересоздаим структуру каталогов
# Если папка существует, то удаляем ее со всеми вложенными каталогами и файлами
if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)

# Создание папки BASE_DIR
os.mkdir(BASE_DIR)

# Сцепляем путь до папки с именем вложенной папки. Аналогично BASE_DIR + '/train'
train_dir = os.path.join(BASE_DIR, 'train')

# Создаем подпапку, используя путь
os.mkdir(train_dir)

# Сцепляем путь до папки с именем вложенной папки. Аналогично BASE_DIR + '/validation'
validation_dir = os.path.join(BASE_DIR, 'validation')

# Создаем подпапку, используя путь
os.mkdir(validation_dir)

# Сцепляем путь до папки с именем вложенной папки. Аналогично BASE_DIR + '/test'
test_dir = os.path.join(BASE_DIR, 'test')

# Создаем подпапку, используя путь
os.mkdir(test_dir)

In [ ]:
dir_names = sorted(list(os.listdir('/content/temp')))
dir_names

['test_set', 'training_set']

In [ ]:
test_files = []
tr_files = []

for i in dir_names:
  for dirpath, dirs, filenames in os.walk('/content/temp'+'/'+i):
    for filename in filenames:
      if i==dir_names[0]:
        test_files.append(os.path.join(dirpath, filename))
      if i==dir_names[1]:
        tr_files.append(os.path.join(dirpath, filename))

print(len(tr_files))
print(len(test_files))

8005
2023


In [ ]:
def create_dataset(
    img_path: str,
    new_path: str,
    class_name: str,
    start_index: int,
    end_index: int):

    src_path = os.path.join(img_path, class_name)  # Полный путь к папке с изображениями класса
    dst_path = os.path.join(new_path, class_name)  # Полный путь к папке с новым датасетом класса

    # Получение списка имен файлов с изображениями текущего класса
    class_files = os.listdir(src_path)

    # Создаем подпапку, используя путь
    os.mkdir(dst_path)

    for fname in class_files[start_index : end_index]:
        src = os.path.join(src_path, fname)
        dst = os.path.join(dst_path, fname)
        shutil.copyfile(src, dst)

for class_label in range(CLASS_COUNT):    # Перебор по всем классам по порядку номеров (их меток)
    class_name = CLASS_LIST[class_label]  # Выборка имени класса из списка имен

    create_dataset(IMAGE_PATH, train_dir, class_name, 0, 3000)
    create_dataset(IMAGE_PATH, validation_dir, class_name, 3000, 4000)
    create_dataset(IMAGE_PATH_1, test_dir, class_name, 0, 2023)

In [ ]:
# Аугментация изображений
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

# Загрузка предобученной модели
base_model = MobileNet(weights='imagenet', include_top=False)

# Добавление новых слоев
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Компиляция модели
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

# Проверка модели на тестовой выборке
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


<ipython-input-9-ad3030aa64c2>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
100/100 [==============================] - 59s 560ms/step - loss: 0.5971 - accuracy: 0.8675 - val_loss: 0.1818 - val_accuracy: 0.9480
Epoch 2/10
100/100 [==============================] - 63s 627ms/step - loss: 0.2306 - accuracy: 0.9135 - val_loss: 0.1249 - val_accuracy: 0.9540
Epoch 3/10
100/100 [==============================] - 61s 608ms/step - loss: 0.1908 - accuracy: 0.9295 - val_loss: 0.1878 - val_accuracy: 0.9340
Epoch 4/10
100/100 [==============================] - 55s 553ms/step - loss: 0.1800 - accuracy: 0.9315 - val_loss: 0.1037 - val_accuracy: 0.9620
Epoch 5/10
100/100 [==============================] - 61s 615ms/step - loss: 0.1614 - accuracy: 0.9395 - val_loss: 0.0861 - val_accuracy: 0.9660
Epoch 6/10
100/100 [==============================] - 55s 548ms/step - loss: 0.1684 - accuracy: 0.9315 - val_loss: 0.1063 - val_accuracy: 0.9650
Epoch 7/10
100/100 [==============================] - 61s 611ms/step - loss: 0.1358 - accuracy: 0.9485 - val_loss: 0.1113 - val_ac

<ipython-input-9-ad3030aa64c2>:59: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)


test acc: 0.9819999933242798
